# Wine MLP — L1 Regularization
Dataset: sklearn wine; Model: 2 hidden layers (64, 32), ReLU; Loss: CrossEntropy (logits). L1 penalty on weights only.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from dataset_wine import get_wine_loaders, set_seed

set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, val_loader, test_loader, input_dim, num_classes = get_wine_loaders(batch_size=64)
input_dim, num_classes

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim: int, num_classes: int, h1: int = 64, h2: int = 32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, num_classes)  # logits
        )
    def forward(self, x):
        return self.net(x)

model = MLP(input_dim, num_classes).to(device)
model

In [ ]:
def accuracy(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            pred = logits.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total

def train_l1(model, train_loader, val_loader, epochs=50, lr=1e-3, l1_lambda=1e-4):
    opt = optim.Adam(model.parameters(), lr=lr)
    ce = nn.CrossEntropyLoss()
    best = {'val_acc': 0.0, 'state': None}
    for ep in range(1, epochs+1):
        model.train()
        running = 0.0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            logits = model(x)
            ce_loss = ce(logits, y)
            l1 = 0.0
            for name, p in model.named_parameters():
                if p.requires_grad and p.dim() > 1:  # weight matrices
                    l1 = l1 + p.abs().sum()
            loss = ce_loss + l1_lambda * l1
            loss.backward()
            opt.step()
            running += loss.item()
        val_acc = accuracy(model, val_loader)
        if val_acc > best['val_acc']:
            best['val_acc'] = val_acc
            best['state'] = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        if ep % 10 == 0:
            print(f'Epoch {ep:02d} | loss={running/len(train_loader):.4f} | val_acc={val_acc:.4f}')
    if best['state'] is not None:
        model.load_state_dict(best['state'])
    return model, best['val_acc']

model, best_val = train_l1(model, train_loader, val_loader, epochs=50, lr=1e-3, l1_lambda=1e-4)
best_val, accuracy(model, test_loader)